Price and returns using Kalman Filter.

Generate prices based on some assumed price dynamics and then try uncover the patterns of speed. acceleration and variances.

$$
\Large
\begin{align}
P_{t+\delta} &= P_{t} e^{\mu_t \delta}\\
L &= log(\frac{P_{t+\delta}}{P_t}) \\
L &= \mu_t \delta
\end{align}
$$

This can be modeled as 

$$
\Large
\begin{align}
L_{t+\delta} &= \mu_t \delta  + \epsilon_1\\
\mu_{t+\delta} &= \mu_t + a_t \delta + \epsilon_2\\
a_{t+\delta} &= a_t + \epsilon_3
\end{align}
$$

We need to be precise
* what is $\mu_t$ ? By above it is compuded annual return or drift


In [25]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [26]:
#! pip install filterpy

In [27]:
np.random.seed(144)

N           = 200
a0          = 0.000
window_size = 20

a_vals = [a0]
eps_3 = np.random.randn(N+window_size)/100000

a_vals = [a0]
for i in range(N+window_size-1):
    a_vals.append(a_vals[-1] +  eps_3[i])

a_vals = np.array(a_vals)
print(f"Length of avals = {len(a_vals)}")
a_vals[0:10]

Length of avals = 220


array([ 0.00000000e+00, -1.29857105e-05, -1.39110990e-05, -1.32103560e-05,
        5.34016584e-06,  1.90427611e-05,  1.72169658e-05,  5.51673181e-06,
        1.57962758e-05,  7.45159769e-06])

In [28]:
weights = np.ones(window_size) / window_size

a_vals_smoothed = np.apply_along_axis(lambda y: sum(weights*y), 
                          1, 
                          np.lib.stride_tricks.sliding_window_view(a_vals, window_size))

In [29]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter( 
                 x=np.array(range(N)),
                 y=a_vals,
                 name = "acceleration"))
fig.add_trace(go.Scatter( 
                 x=np.array(range(N)),
                 y=a_vals_smoothed,
                 name = "smoothed acceleration"))
fig.show()

##### DRIFT term

$$
\mu_{t+\delta} = \mu_t + a_t \delta + \epsilon_2
$$


In [34]:
eps_2 = np.random.normal(loc=0.0, scale = 0.0001, size = N)

delta = 1/260
yearly_mu0 = 0.06
yearly_mus = [yearly_mu0]
for i in range(N):
    yearly_mus.append(yearly_mus[-1] + a_vals_smoothed[i]*delta + eps_2[i])
    
yearly_mus = np.array(yearly_mus)

In [35]:
fig = go.Figure()
fig.add_trace(go.Scatter( 
                 x=np.array(range(N)),
                 y=yearly_mus,
                 name = "mu values"))
fig.show()

$$
\Large
\begin{align}
L_{t+\delta} &= \mu_t \delta + \epsilon_1\\
\mu_{t+\delta} &= \mu_t + a_t \delta + \epsilon_2\\
a_{t+\delta} &= a_t + \epsilon_3
\end{align}
$$


In [30]:
# yearly_mus

In [16]:
## Get L_t values
eps_1 = np.random.randn(N+1)/100

Lts = yearly_mus * delta + eps_1 

Lts[0:10]

array([ 0.00765945, -0.01077232,  0.0079932 ,  0.00973288,  0.00414529,
       -0.00803603, -0.00584462, -0.00858677, -0.01187843, -0.006738  ])

In [17]:
price_0 = 10.0
prices = price_0 * np.exp(np.cumsum(Lts))

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter( 
                 x=np.array(range(N)),
                 y=prices,
                 name = "Prices"))
fig.show()

#### We have data now - we need to fit Kalman using our model

$$
\Large
\begin{align}
L_{t+\delta} &= \mu_t \delta + \epsilon_1\\
\mu_{t+\delta} &= \mu_t + a_t \delta + \epsilon_2\\
a_{t+\delta} &= a_t + \epsilon_3
\end{align}
$$








Writing as measurement and transition equations:

$$
\Large
\begin{align}
\Delta L_{t+\delta} &= \begin{bmatrix}
                \delta & 0
              \end{bmatrix} \begin{bmatrix} \mu_t \\ a_t \end{bmatrix} + \epsilon_1\\
\begin{bmatrix} \mu_t \\ a_t \end{bmatrix} &=  \begin{bmatrix}
                                                    1 & \delta\\
                                                    0 & 1
                                                \end{bmatrix} \begin{bmatrix} \mu_{t-1} \\ a_{t-1} \end{bmatrix}   +  \epsilon  
\end{align}
$$

Assume We dont know $\mu_t, a_t$ and these are the hidden states we need to find out so that we can do predict and check. 

In [36]:
#1. start with theta0 as 
mu_guess = 0.0
acc_guess = 0.0

# How to estimate W, V ? using some MLE - 
W_t = np.diag([0.0001,0.0001])
V_t = 0.00001
F_t = np.matrix([delta , 0]).reshape(-1,2)

m0 = np.matrix([mu_guess, acc_guess]).reshape(2,-1)
m_prev = m0

#100 mean low confidence
C0 = np.matrix([[100, 0], [0, 100]])
C_prev = C0
G = np.matrix([[1.0, delta], [0, 1.0]])

posterior_states = []

for t in range(N-1):
    #2. One step forecast for the state
    #print(f"G: {G.shape} m_prev: {m_prev.shape}")
    a_t = G @ m_prev
    R_t = G @ C_prev @ G.T + W_t

    #3. One step forecast for the observation
    f_t = F_t@a_t
    Q_t = F_t @ R_t @ F_t.T + V_t

    #4. Compute Posterior
    y_t = delta_prices[t]
    #print(f"a_t: {a_t.shape} R_t: {R_t.shape} f_t:{f_t.shape} ")
    Q_t_inv = np.linalg.inv(Q_t)
    m_t = a_t + R_t @ (F_t.T) @ Q_t_inv * (y_t - f_t)
    C_t = R_t - R_t @ (F_t.T) @ Q_t_inv @ F_t @ R_t
    m_prev = m_t
    C_prev = C_t
    posterior_states.append(np.ravel(m_t))

In [21]:
posterior_mu = [m0[0,0]]
posterior_mu.extend([x[0] for x in posterior_states])
posterior_a = [m0[1,0]]
posterior_a.extend([x[1] for x in posterior_states])

In [22]:
len(posterior_mu)

200

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.array(range(N)), y = yearly_mus, name = "mu_s"))
fig.add_trace(go.Scatter(x = np.array(range(N)), y = posterior_mu, name = "posterior_mu_s"))
fig.show()

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.array(range(N)), y = a_vals, name = "a_vals"))
fig.add_trace(go.Scatter(x = np.array(range(N)), y = posterior_a, name = "posterior_a"))
fig.show()